In [ ]:
# import pandas
import pandas as pd

In [ ]:
#Google Patents Search https://patents.google.com/?q=crispr&country=US&status=GRANT&language=ENGLISH&type=PATENT
#download results as gp-search-20200124-065711.csv
#24 JAN 2020

In [ ]:
#start with a download of patent search with URLs to patent data
df = pd.read_csv('gp-search-20200124-065711.csv', names = ['id', 'title', 'assignee', 'inventor/author', 
                                           'priority date', 'filing/creation date', 
                                           'publication date', 'grant date', 'result link', 
                                           'representative figure link'])[2:]

In [ ]:
#reset dataframe index
df = df.reset_index(drop=True)
df.head()

In [ ]:
#create new columns to fill later with claim and classification data
df['claim'] = ""
df['classifications'] = ""
df.head()

In [ ]:
#save URLs to patent data to a list, for retrieval
urls = df['result link'].tolist()
urls

In [ ]:
#check to make sure that the list of URLs is == to the list of data, should match
len(urls) == len(df)

Using the URL list, scrape Google Patents for claims and classification data for each patent. 

In each iteration, append the data to a new column/row in the dataframe, df. 

May take hours to run. Please use the CSV file with raw text data if testing the scraper is not your objective. Located in repo. 

In [ ]:
%%time

from selenium import webdriver
from time import sleep

driver = webdriver.Chrome('/Users/cheese/lData/PatentNLP/chromedriver')

query_claims = '//*[@id="claims"]'
query_classes = '//*[@id="classifications"]'

df_claims = pd.DataFrame()

# create an empty list "claims"
claims = []
# create an empty list "classes"
classes = []
# create an empty list "classes"
anticipexpirations = []


# create a for loop for computer to read through the list of urls 
for num, url in enumerate(urls, start=0):
    # tell driver to get the url that the computer is reading
    driver.get(url)
    # tell chromedriver to find the claims in the pages that the driver is getting
    
    
    print("url {}: {}".format(num, url))
    description_claim = driver.find_elements_by_xpath(query_claims)
    # create a for loop (within the previous for loop) (*avoid nested for loops whenever possible!) 
    # to have computer look through each page for claims
    for i in description_claim:
        # if claims exist for the patent, add that to the empty list of claims
        if i:
            #create a list of claims based on each URL
            #claims.append(i.text)
            
            #AND/OR write each claim to the data frame as you go
            claim_text = i.text
            df.at[num,'claim']=claim_text
        # if no claims found, add "none"
        else:
            #write a value of 'NO-RECORD-RETRIEVED' if none is retrieved
            #claims.append('NO-RECORD-RETRIEVED')
            
            #AND/OR write each claim to the data frame as you go
            df.at[num,'claim']='NO-RECORD-RETRIEVED'
    
    # tell chromedriver to find the classes in the pages that the driver is getting
    description_classes = driver.find_elements_by_xpath(query_classes)
    # create a for loop (within the previous for loop) (*avoid nested for loops whenever possible!) 
    # to have computer look through each page for claims
    for i in description_classes:
        # if classes exist for the patent, add that to the empty list of classes
        if i:
            #classes.append(i.text)
            classes_text = i.text
            df.at[num, 'classifications']=classes_text
        # if no classes found, add "none"
        else:
            #classes.append('None')
            df.at[num, 'classifications']='NO-RECORD-RETRIEVED'
            
    # let the driver pause for 0.5 seconds between each page
    sleep(0.25)
# stop the driver
print("done")
driver.quit()

In [ ]:
#check the table for results
df.head()

In [ ]:
#save the scraped data to csv to avoid re-scraping
df.to_csv('gp-search-20200124-065711-raw_with_claims.csv')